# Seminar Quantitativ Microbiology 1: 
# Simulation of Genome Scale Metabolic Models with CobraPy

## Introduction
The seminar provides a guide of how to work with genome scale metabolic models (GSMM) of micro-organisms. This seminar extends the introduction to CobraPy from the seminar in Quantitative Microbiology 1. The goal of this seminar is to identify minimal medium composition, extract information about and selecting the appropriate biomass composition formula and testing the reproduction of experimental data by the GSMM. We examine a recent model of *Komagataella phaffii* (*P. pastoris*) ([Tomas-Gamisans et al., 2017](https://dx.doi.org/10.1111/1751-7915.12871)). This organism is of biotechnological relevance because it can glycosylate recombinant proteins for use as drugs and it can metabolize methanol as a potential alternative to petrol based chemistry ([Liebal et al., 2018](https://dx.doi.org/10.1016/j.mec.2018.e00075)).

We will analyse the GSMM of *K. phaffii* to estimate for the exchange reactions the range of permissible flux values. This indicates which phenotypes we can expect during standard cultivation and can highlight easily overproduced metabolites. Subsequently, we will identify the minimal medium composition. An important property of GSMM is their ability to predict growth rates. We will extract experimentally measured growth rates for various substrates and compare them with predictions of the model. The data reproduction is exemplified with the substrates of methanol and glycerol and the self-learning task is to supplement growth rates based on glucose uptake rate from literature ([Lehnen et al., 2017](https://dx.doi.org/10.1016/j.meteno.2017.07.001)).

The seminar uses Jupyter notebooks, a new way to use and visualize code in the web. Such a notebook is composed of a sequence of cells. Cells can be either text/comments, like this introduction, or it contains python code to be run. The output for each code-cell is shown directly below it. For a overview on Jupyter notebooks read [this review](https://www.nature.com/articles/d41586-018-07196-1).


---

## Tutorial Steps
  * Set up of Python environment
    * Basic libraries(sys, pandas, numpy, matplotlib, zipfile, cobrapy)
  * Analysis of Genome Scale Metabolic Model
    * Retrieval of GSMM for *K. phaffii*
    * Flux variability of exchange reactions
    * Minimal medium composition
  * Experimental growth rate reproduction
    * Familiarizing with biomass composition reactions
    * Defining functions for correct biomass equation switch
    * Data retrieval
    * Simulation loop
    * Graphical output


## Set-up compute environment

Before we can analyse GSMM, we have adjust the python environment that it integrates the cobrapy toolbox and downloading the GSMM.

### Basic Python libraries 
Some libraries that facilitate data manipulation

In [1]:
# Adapting environment to GitHub or Google Colab
# In GitHub, the full repository is available with all directories, and libraries can be installed with pip on requirements.txt
# In Google Colab only the selected notebook is loaded and the requirements file must be downloaded from GitHub before installation.
# 

# file system and path operations
import os
import sys
if 'google.colab' in sys.modules:
    IN_COLAB = True
    # Download the requirements file
    os.system('wget https://raw.githubusercontent.com/biolabsim/BioLabSim/refs/heads/master/requirements.txt')
    os.system('pip install -r requirements.txt')
else:
    IN_COLAB = False
    os.system('pip install -r ../requirements.txt')

print('Environment ready.')

Environment ready.


In [2]:
# file system and path operations
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.utils import io

# load wget and install it if necessary
import wget
from cobra.io import read_sbml_model

import zipfile


print('Done')

Done


## Analysis of Genome Scale Metabolic Model

### Retrieval of GSMM for *K. phaffii*

*K. phaffii* is an important biotechnological organism and several GSMMs have been generated. We will use the model generated by [Tomas-Gamisans et al., 2017](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5743807/#!po=67.8571). Please click on the link and find the additional data file for the model iMT1026v3 in the chapter "Support Information", right click on "Click here for additional data file" and select "Copy link address". In the next cell, replace the *None* after "!wget" with the link address.

The zip-file "MBT2-11-224-s003.zip" with the SBML model of *K. phaffii* is unzipped into a new folder called `folder`. 

**Task**:
 - Replace the *None* with the web adress of the supplementary for the SBML model iMT1026v3 (use web adress without quotes)

In [ ]:
!wget None

with zipfile.ZipFile('MBT2-11-224-s003.zip', 'r') as zip_ref:
  zip_ref.extractall()

print('File download and zip extraction done.')

### SBML extraction from zip-file
Load the SBML model of K. phaffii located at `folder/mbt212871-sup-0003-AppendixS3.xml` as a cobrapy mode.

**Task**:
 - Replace the *None* with the SBML file location

In [ ]:
# suppress output, because hundreds of warnings are generated
with io.capture_output() as captured:
  # generating cobra variable from SBML/xml file
  model = read_sbml_model('None'); # 
print('model loaded!')   
model

### Flux variability of exchange reactions

Flux balance analysis provides a single optimal solution. Mostly, there exist a number of alternative flux distributions around the optimum, which can be physiologically relevant. To identify the variability of exchange fluxes around the optimum solution 'flux variability analysis' can be performed ([Mahadevan & Schilling, 2003](http://dx.doi.org/10.1016/j.ymben.2003.09.002)). Use the following command to identify minimum and maximum flux ranges of the model for the exchange reactions.

In [ ]:
model.summary(fva=.95) # additional argument specifies allowed deviation from the optimum

### Media test
The availability of nutrients has a major impact on metabolic fluxes and cobrapy provides some helpers to manage the exchanges between the external environment and the metabolic model. More detailed descriptions: [here](https://cobrapy.readthedocs.io/en/latest/media.html)

In [ ]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth, minimize_components=True)


## Experimental growth rate reproduction

### Familiarizing with biomass composition reactions

Microorganisms adapt to their substrate. Different substrates provide different energy content and require different cellular resources to become metabolized. In GSMM these differences may be represented by different equations/reactions for the substrates. In iMT1026 for *K. phaffii* there are various biomass equations for glucose, glycerol, glucose-glycerol mixtures, and methanol. When simulating a model, we have to make sure we use the right biomass equation fitting with the substrate.

The output of the biomass search will generate a long list of different reactions. Specific reactions for different substrates may be needed if the biomass composition changes. The biomass composition might also change in response to different substrate uptake rates. Some of the biomass equations in the model are based on different relative uptake rates of glycerol and methanol (e.g., 80/20, 60/40, 40/60).

**Task**:
 - Replace the *None* with 'BIOMASS'

In [ ]:
# List of all reactions with 'BIOMASS' in their name
model.reactions.query(None) # 

Looking in detail to biomass with methanol; use the reaction name given to you from the list of all biomass reactions.

**Task**:
 - Replace the *None* with the reaction name for methanol-based biomass accumulation.

In [ ]:
model.reactions.get_by_id(None) # 

### Defining functions for correct biomass equation switch

For each substrate, the boundary exchange fluxes are activated and the reactions of competing substrates are disabled. For each substrate a dedicated biomass equation is available in the model. Add the correct substrate-specific biomass equation ID in the functions below.

**Task**
 - Replace the *None* with the reaction id of the corresponding substrate of the function

In [ ]:
def AdaptGlucose(model, glc_up):
  model.objective = 'BIOMASS'
  # setting uptake reactions right
  model.reactions.Ex_meoh.lower_bound = 0
  model.reactions.Ex_glyc.lower_bound = 0
  model.reactions.Ex_glc_D.lower_bound = -np.abs(glc_up)
  # setting additional biomass composition
  model.reactions.LIPIDS.upper_bound = 1000
  model.reactions.PROTEINS.upper_bound = 1000
  model.reactions.STEROLS.upper_bound = 1000
  model.reactions.BIOMASS.upper_bound = 1000  
  # deactivating Glyc-based biomass composition
  model.reactions.LIPIDS_glyc.upper_bound = 0
  model.reactions.PROTEINS_glyc.upper_bound = 0
  model.reactions.STEROLS_glyc.upper_bound = 0
  model.reactions.BIOMASS_glyc.upper_bound = 0  
 # deactivating meoh-based biomass composition
  model.reactions.LIPIDS_meoh.upper_bound = 0
  model.reactions.PROTEINS_meoh.upper_bound = 0
  model.reactions.STEROLS_meoh.upper_bound = 0
  model.reactions.BIOMASS_meoh.upper_bound = 0
  return model

def AdaptMethanol(model, meoh_up):
  model.objective = None
  # setting uptake reactions right
  model.reactions.Ex_glc_D.lower_bound = 0
  model.reactions.Ex_glyc.lower_bound = 0
  model.reactions.ATPM.lower_bound = .4
  model.reactions.Ex_meoh.lower_bound = -np.abs(meoh_up)
  # setting additional biomass composition
  model.reactions.LIPIDS_meoh.upper_bound = 1000
  model.reactions.PROTEINS_meoh.upper_bound = 1000
  model.reactions.STEROLS_meoh.upper_bound = 1000
  model.reactions.BIOMASS_meoh.upper_bound = 1000
  # deactivating Glyc-based biomass composition
  model.reactions.LIPIDS_glyc.upper_bound = 0
  model.reactions.PROTEINS_glyc.upper_bound = 0
  model.reactions.STEROLS_glyc.upper_bound = 0
  model.reactions.BIOMASS_glyc.upper_bound = 0  
  # deactivating Glc-based biomass composition
  model.reactions.LIPIDS.upper_bound = 0
  model.reactions.PROTEINS.upper_bound = 0
  model.reactions.STEROLS.upper_bound = 0
  model.reactions.BIOMASS.upper_bound = 0  
  return model

def AdaptGlycerol(model, glyc_up):
  model.objective = None
  # setting uptake reactions right
  model.reactions.Ex_meoh.lower_bound = 0
  model.reactions.Ex_glc_D.lower_bound = 0
  model.reactions.ATPM.lower_bound = 2.5
  model.reactions.Ex_glyc.lower_bound = -np.abs(glyc_up)
  # setting additional biomass composition
  model.reactions.LIPIDS_glyc.upper_bound = 1000
  model.reactions.PROTEINS_glyc.upper_bound = 1000
  model.reactions.STEROLS_glyc.upper_bound = 1000
  model.reactions.BIOMASS_glyc.upper_bound = 1000  
  # deactivating MeOH-based biomass composition
  model.reactions.LIPIDS_meoh.upper_bound = 0
  model.reactions.PROTEINS_meoh.upper_bound = 0
  model.reactions.STEROLS_meoh.upper_bound = 0
  model.reactions.BIOMASS_meoh.upper_bound = 0
  # deactivating Glc-based biomass composition
  model.reactions.LIPIDS.upper_bound = 0
  model.reactions.PROTEINS.upper_bound = 0
  model.reactions.STEROLS.upper_bound = 0
  model.reactions.BIOMASS.upper_bound = 0
  return model


print('Functions defined.')

### Data retrieval

For evaluation of the growth rate prediction of the *K. phaffii* model we use experimental data from the closely related organism *Ogataea polymorpha*. The measurements in the table are extracted from [van Dijken et al. 1976](https://dx.doi.org/10.1007/bf00446560) for methanol and from [de Koning et al., 1987](https://dx.doi.org/10.1007/BF00456710) and [Moon et al., 2003](https://dx.doi.org/10.1385/ABAB:111:2:65). The data file is already available as csv `../Resources/Opol-expt-grwth_MeOH-Glyc.csv`.

**Task**: 
 - Replace the None with the csv file location. 
 - (In **Google-Colab** use `wget` with the github file link `https://github.com/biolabsim/BioLabSim/raw/master/Resources/Opol-expt-grwth_MeOH-Glyc.csv`)

In [ ]:
data = pd.read_csv('None', delimiter=',|;', engine='python') #
data

### Adding new measurements
We have conducted experiments with *O. polymorpha* wich we want to add to the table. The measurements are based on glucose and reported in [Lehnen et al., 2017](https://doi.org/10.1016/j.meteno.2017.07.001). The required information is the growth rate on glucose for *O. polymorpha*. Extract the necessary information from Table 3 in the article. The reaction name for the 'Exchange'-reaction is 'Ex_glc_D', replace this in the corresponding position of `None` in the cell below.

**Task**:
 - Scan the paper [Lehnen et al., 2017](https://doi.org/10.1016/j.meteno.2017.07.001) for the glucose growth rate.
 - Replace the first *None* with the reaction ID for Glucose, the second *None* with the glucose uptake rate, the third *None* with the growth rate. (Important: Reaction name in quotes, numbers without quotes)
 - Alternative solution: 
   - Download the data file `data/Opol-expt-grwth_MeOH-Glyc.csv`
   - Add a row with the Lehnen data
   - Upload the data file
   - Reload the data into the `data` variable with the previous cell

In [ ]:
data = data.append({'Substrate':'Glucose', 'Exchange':'None', 'uptake rate (mmol/gCDW/h)':None, 'growth rate (/h)':None, 'source':'Lehnen et al.'}, ignore_index=True)

### Simulation loop
For-Loop over all experimental data points.

Remember to add a decision when you include glucose.

In [ ]:
growth_simulated = [];
# iteration over all rows in 'data'
for index, row in data.iterrows():
  with model as test_model:
    print(index) # printing the row number to get feedback that everything is working
    # selecting the right substrate in the model based on 'Substrate' in 'data'
    if row['Substrate'] == 'Methanol':
      model = AdaptMethanol(test_model, row['uptake rate (mmol/gCDW/h)'])
    elif row['Substrate'] == 'Glycerol':
      model = AdaptGlycerol(test_model, row['uptake rate (mmol/gCDW/h)'])
    elif row['Substrate'] == 'Glucose':
      test_model = AdaptGlucose(test_model, row['uptake rate (mmol/gCDW/h)'])
    else:
      print('substrate not considered')      
    growth_simulated.append(model.slim_optimize())

### Graphical output

**Tasks**:
 - Add the right axis labels, and a file name.

In [ ]:
plt.scatter(data['growth rate (/h)'][0:7], growth_simulated[0:7], s=50, c='k', marker='o');
plt.scatter(data['growth rate (/h)'][7], growth_simulated[7], s=50, c='k', marker='s');
plt.scatter(data['growth rate (/h)'][8], growth_simulated[8], s=50, c='k', marker='d');
plt.xlabel('None');
plt.ylabel('None');
myline = np.linspace(0,np.max(growth_simulated),10);
plt.plot(myline,myline,'k--');
plt.title('Growth rate comparison');
plt.legend(['Optimum', 'Methanol (van Dijken)', 'Glycerol (deKoning, Moon)', 'Glucose (Lehnen et al.)'], loc=2);
plt.style.use('seaborn-paper')

# Saving figure
plt.savefig('None.png')